## Object Following (50 pts)

In this notebook we'll show how you can follow an object with the crazyflie!  We'll use a pre-trained neural network that was trained on the [COCO dataset](http://cocodataset.org) to detect 90 different common objects.  These include

* Person (index 0)
* Cup (index 47)

and many others (you can check [this file](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt) for a full list of class indices). We use the MobileNet SSD (Single Shot Detector) trained on the COCO dataset. SSD models are often faster than other detection models and the MobileNet backbone is less computationally intensive, so this will help for real-time execution! The model is sourced from the [TensorFlow object detection API](https://github.com/tensorflow/models/tree/master/research/object_detection),
which provides utilities for training object detectors for custom tasks also!

We won't run through all of the training and optimization steps in this notebook though. The goal here is to demonstrate what one can do with neural networks. In the final project, you will get a chance to train neural networks for obstacle avoidance and navigation. 

Anyways, let's get started!  First, we will load the pre-trained network. Make sure to have the Lab8_Supplement directory downloaded. Also download the model and place in the Lab8_Supplement directory [https://drive.google.com/file/d/1vIS9XySf5kdmVqPCtCpHG_-FL6RB8oOP/view](https://drive.google.com/file/d/1vIS9XySf5kdmVqPCtCpHG_-FL6RB8oOP/view).

### Compute detections on single camera image

For this lab, we will be using OpenCV's DNN module which provides us with functionalities for deep learning inference. You can read more about how we are using it for [object detection](https://learnopencv.com/deep-learning-with-opencvs-dnn-module-a-definitive-guide/). Specifically, we can load in the MobileNet SSD network that was trained using the Tensorflow framework. OpenCV's DNN module allows for multi-framework use (e.g., PyTorch and Caffe).

First, we load in the COCO class names (e.g., person, potted plant, etc.), assign colors to the classes (this is useful for visualizing bounding boxes), and load the weights of the pre-trained neural network. 

In [1]:
import cv2
import numpy as np

# load the COCO class names
with open('Lab8_Supplement/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')
    
# get a different color array for each of the classes
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# load the DNN model
model = cv2.dnn.readNet(model='Lab8_Supplement/frozen_inference_graph.pb',
                        config='Lab8_Supplement/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt', 
                        framework='TensorFlow')

Now we will prepare an image for object detection with our model. `blobFromImage()` prepares the image into the correct format for our model. Specifically, we resize our input image to 300x300 and normalize the RGB channels with the mean parameter. Then we forward propagate the image through the model to obtain the detections. Each detection is of the form ( _, class_id, confidence, box_x, box_y, box_width, box_height) where box_x, box_y, box_width, box_height provide information for creating the bounding box of around the detected object.

In [2]:
# read the image from disk
image = cv2.imread('Lab8_Supplement/Lab8_image.jpg')
image_height, image_width, _ = image.shape

# create blob from image
blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(104, 117, 123), 
                             swapRB=True)

# create blob from image
model.setInput(blob)

# forward pass through the model to carry out the detection
detections = model.forward()

Next we visualize the detections. You should see a bounding box, classification, and confidence value appear around each COCO object (potted plant and cup).

In [3]:
# loop over each of the detection
for detection in detections[0, 0, :, :]:
    # extract the confidence of the detection
    confidence = detection[2]
    # draw bounding boxes only if the detection confidence is above...
    # ... a certain threshold, else skip
    if confidence > .4:
        # get the class id
        class_id = detection[1]
        # map the class id to the class
        class_name = class_names[int(class_id)-1]
        color = COLORS[int(class_id)]
        # get the bounding box coordinates
        box_x = detection[3] * image_width
        box_y = detection[4] * image_height
        # get the bounding box width and height
        box_width = detection[5] * image_width
        box_height = detection[6] * image_height
        # draw a rectangle around each detected object
        cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=1)
        # put the FPS text on top of the frame
        text = class_name + ' ' + '%.2f' % (confidence)
        cv2.putText(image, text, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, color, 1)

while(True):
    cv2.imshow('image', image)
    
    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cv2.imwrite('Lab8_Supplement/image_result.jpg', image)


True

To print just the first object detected in the example image, we could call the following:

In [4]:
object_number = 0
print(detections[0, 0, object_number, :])

[ 0.         64.          0.886291    0.38060737  0.31562248  0.6190595
  0.71215916]


### Compute detections on a live video feed

The following cell will perform the same object detection and labeling on a live feed from your CrazyFlie camera! Note that the drone will not fly, you are simply using the camera. This should give you a sense of appropriate distances for detection, as well as the confidence for the detection of different objects from the Coco dataset.

In [4]:
# This may open your webcam instead of the CrazyFlie camera! If so, try
# a different small, positive integer, e.g. 1, 2, 3.
camera = 0
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    image_height, image_width, _ = frame.shape

    # create blob from image
    blob = cv2.dnn.blobFromImage(image=frame, size=(300, 300), mean=(104, 117, 123), 
                                 swapRB=True)

    # create blob from image
    model.setInput(blob)

    # forward pass through the model to carry out the detection
    detections = model.forward()

    # loop over each of the detection
    for detection in detections[0, 0, :, :]:
        # extract the confidence of the detection
        confidence = detection[2]
        # draw bounding boxes only if the detection confidence is above...
        # ... a certain threshold, else skip
        if confidence > .4:
            # get the class id
            class_id = detection[1]
            # map the class id to the class
            class_name = class_names[int(class_id)-1]
            color = COLORS[int(class_id)]
            # get the bounding box coordinates
            box_x = detection[3] * image_width
            box_y = detection[4] * image_height
            # get the bounding box width and height
            box_width = detection[5] * image_width
            box_height = detection[6] * image_height
            # draw a rectangle around each detected object
            cv2.rectangle(frame, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=1)
            # put the FPS text on top of the frame
            text = class_name + ' ' + '%.2f' % (confidence)
            cv2.putText(frame, text, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, color, 1)
    
    # Compute
    cv2.imshow('frame', frame)    

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

### Control robot to follow central object

Now we want our robot to follow an object of the specified category (e.g., person, etc.).  To do this we'll do the following

1.  Detect objects matching the specified class
2.  Select object closest to center of camera's field of vision; this is the 'target' object
3.  Control the robot towards target object; otherwise hover

We'll also create a controller that will use the distance between the target object and the center of the robot's field of view to follow the object as well as use the bounding box size to determine when to stop. 

First, let's define some functions that will process the images from the crazyflie. 

### Task 1 (10 pts) ###

Fill in the function "closest_detection" below. This should find the detected object that is closest to the center of the image. 

In [5]:
import time

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    center_x = (detection[3] + detection[5]) / 2.0 - 0.5
    center_y = (detection[4] + detection[6]) / 2.0 - 0.5
    return (center_x, center_y)

def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """TODO: Find the detection closest to the image center"""
    # Loop through and find the detection that is closest to the image center
    # You can use the detection_center function above to find the center of the detected object
    # Note that the origin (i.e., (x,y) = (0,0)) corresponds to the center of the image. So you can
    # use the "norm" function above to find the detection that is closest to the center.
    # Return the det that corresponds to the closest detection to the image center.
    # If nothing is detected, return None.
    
    min_dist = np.inf
    champ_det = None

    for det in detections:

        cur_dist = norm(detection_center(det))

        if cur_dist < min_dist:
            champ_det = det
            min_dist = cur_dist

    return champ_det

Great, now let's get ready to control the crazyflie to follow an object! Below are a few functions to help move the crazyflie.

In [6]:
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander


def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            # print("{} {} {}".
            #     format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)    
    return

# Ascend and hover:
def ascend_and_hover(cf):
    # Ascend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)
    # Hover at 1 meter:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    return

def hover(cf):
    print('Hovering:')
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    return
    
# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 10)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

### Task 2 (20 pts) ###

Fill in the controller below that says "TODO" to make the crazyflie follow the object. The controller should use the inputs to keep the detected target in the center of its view as well determine when to stop (send True flag) so that the crazyflie stops and lands before crashing into the tracked object. (Note: the execution code implements the actual stopping using the flag).

In [9]:
def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
    """
    
    cf: crazyflie instance
    box_x: x coordinate of the center of the bounding box in the image
    box_y: y coordinate of the center of the bounding box in the image
    box_width: width of the bounding box in the image
    box_height: height of the bounding box in the image
    x_cur: current x position
    y_cur: current y position
    
    Return True to indicate that the drone is close to the target and thus exit the loop to stop and descend, new x, new y
    Return False to indicate continuing to follow the target, new x, new y.
    
    """
    
    #### TO DO: Fill below ####
    # Exit condition/method using size of the bounding box
    size_limit = 0.9

    if min(box_width, box_height) >= size_limit:
        print("too close to the target!")
        print(f"box_width: {box_width}, box_height: {box_height}")
        return True, x_cur, y_cur

    #### TO DO: Fill below ####
    # Determine the x and y velocity
    kx = 5e-1
    ky = 2e-1

    vx = 1
    vy = -box_x
    
    x_command = x_cur + kx * vx
    y_command = y_cur + ky * vy
    
    # Set velocity
    cf.commander.send_position_setpoint(x_command, y_command, 1, 0) # Do not edit this line
    
    return False, x_command, y_command
    

The following code will test your controller on the crazyflie. There are several parameters at the top that may be useful to change as indicated, otherwise do not modify the code. Please read the safety and submission instructions below before running.

In [11]:
import cv2
import time
import numpy as np

# load the COCO class names
with open('Lab8_Supplement/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')

# get a different color array for each of the classes
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# load the DNN model
model = cv2.dnn.readNet(model='Lab8_Supplement/frozen_inference_graph.pb',
                        config='Lab8_Supplement/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt', 
                        framework='TensorFlow')

# ************ Parameters that might be useful to change ************ 
# COCO label id that we want to track
tracking_label = 62 # PERSON (1), CHAIR (62), BACKPACK (27)

# Set the URI the Crazyflie will connect to
group_number = 3
uri = f'radio://0/{group_number}/2M'

# Possibly try 0, 1, 2 ...
camera = 0

# Confidence of detection
confidence = 0.4

# ******************************************************************

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascend to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        # capture the video
        cap = cv2.VideoCapture(camera)
        
        # get the video frames' width and height
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False

        # Ascend and hover a bit
        set_PID_controller(cf)
        ascend_and_hover(cf)
        time.sleep(1)
        
        x_cur = 0
        y_cur = 0
        
        # detect objects in each frame of the video
        while cap.isOpened() and not exit_loop:
            
            # Try to read image
            ret, frame = cap.read()
            if ret:
                image = frame
                image_height, image_width, _ = image.shape

                # create blob from image
                blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(104, 117, 123), 
                                             swapRB=True)

                # forward propagate image
                model.setInput(blob)
                detections = model.forward()

                # select detections that match selected class label
                matching_detections = [d for d in detections[0, 0] if d[1] == tracking_label]

                # select confident detections
                confident_detections = [d for d in matching_detections if d[2] > confidence]

                # get detection closest to center of field of view and draw it
                det = closest_detection(confident_detections) # This relies on the function you wrote above
                
                if det is not None:
                    # get the class id
                    class_id = det[1]
                    # map the class id to the class 
                    class_name = class_names[int(class_id)-1]
                    color = COLORS[int(class_id)]
                    # get the bounding box coordinates
                    box_x = det[3] * image_width
                    box_y = det[4] * image_height
                    # get the bounding box width and height
                    box_width = det[5] * image_width
                    box_height = det[6] * image_height
                    # draw a rectangle around each detected object
                    cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=2)
                    # put the class name text on the detected object
                    cv2.putText(image, class_name, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

                # If nothing is detected, hover
                if det is None:
                    print('no detection...hovering')
                    hover(cf)

                # otherwise  move towards target
                else:
                    print('detection...tracking')
                    _, _, _, box_x, box_y, box_width, box_height = det
                    box_x, box_y = detection_center(det)
                    # addition
                    print(f"box_x: {box_x}, box_y: {box_y}, box_width: {box_width}, box_height: {box_width}")
                    exit_loop, x_cur, y_cur = controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur)

                # Check image
                # cv2.imshow('image', image)
                # if cv2.waitKey(10) & 0xFF == ord('q'):
                #     break
                    
            else:
                print('no image!!')
                
        cap.release()
        
        # Descend and stop all motion:
        hover_and_descend(cf)
        
        cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...


Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'


Crazyflies found:
radio://0/1/2M
radio://0/15/2M
radio://0/1/2M
radio://0/3/2M
radio://0/15/2M
radio://0/1/2M
radio://0/3/2M
radio://0/15/2M
Initializing PID Controller
Waiting for estimator to find position...
no detection...hovering
Hovering:
no detection...hovering
Hovering:
detection...tracking
box_x: 0.007425904273986816, box_y: 0.16384214162826538, box_width: 0.6003879308700562, box_height: 0.6003879308700562
detection...tracking
box_x: -0.2950230538845062, box_y: 0.1319882869720459, box_width: 0.2788941562175751, box_height: 0.2788941562175751
detection...tracking
box_x: 0.005880415439605713, box_y: 0.15653187036514282, box_width: 0.5979256629943848, box_height: 0.5979256629943848
no detection...hovering
Hovering:
no detection...hovering
Hovering:
no detection...hovering
Hovering:
no detection...hovering
Hovering:
no detection...hovering
Hovering:
no detection...hovering
Hovering:
no detection...hovering
Hovering:
no detection...hovering
Hovering:
no detection...hovering
Hoverin

If the previous cell has an error or you lose connection with your drone, run the following cell and restart the kernel.

In [ ]:
cap.release()

# Submission #

Please submit to Gradescope "HW8: Coding" a zip including: this notebook Lab8 (30pts), two videos (20pts see below), and Lab9 notebook (50pts).

For videos, please submit the following:
- (10 pts) A video (e.g., taken from your cellphone) showing the crazyflie following you (or any other person). The person should be moving such that it is clear the crazyflie is changing its tracking to follow the person. Read safety instructions below before trying! The crazyflie should stop and land when close to the person.
- (10 pts) A video showing the crazyflie moving towards a different object (i.e., not a person). For this, you will have to change the "tracking label" in the code above to correspond to the object you want the crazyflie to follow/move towards. You are welcome to choose any object that is convenient for you. For example, you can place a chair (or whatever object you choose) in front of the crazyflie and demonstrate that your code makes the crazyflie move towards that object. The crazyflie should stop and land when close to the object without running into it.

# Safety #

As always, please wear your safety glasses when working with the crazyflie. 

Additionally, for human tracking, please stand OUTSIDE of the netted test space. The drone's camera is capable of detecting people standing behind the net. 